In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, StopWordsRemover, HashingTF, IDF, ChiSqSelector
from pyspark.ml.feature import Tokenizer, Normalizer, StringIndexer
from pyspark.sql.functions import col, lower

from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from pyspark.mllib.util import MLUtils

In [2]:
partial_dataset = "/user/dic24_shared/amazon-reviews/full/reviews_devset.json"
full_dataset = "/user/dic24_shared/amazon-reviews/full/reviewscombined.json"


In [3]:
#Initialize Spark Session
spark = SparkSession.builder \
    .appName("Text_Classification") \
    .getOrCreate()


try:
    df = spark.read.json(partial_dataset)
    print("File read successfully.")
    
    # Show the schema and some data
    df.printSchema()
    df.show(5)
    
except Exception as e:
    print(f"Error: {e}")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
24/05/25 11:33:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/25 11:33:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/25 11:33:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/25 11:33:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/05/25 11:33:42 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/05/25 11:33:4

File read successfully.
root
 |-- asin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)

+----------+--------------------+-------+-------+--------------------+-----------+--------------+--------------------+-------------------+--------------+
|      asin|            category|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|            summary|unixReviewTime|
+----------+--------------------+-------+-------+--------------------+-----------+--------------+--------------------+-------------------+--------------+
|0981850006|Patio_Lawn_and_Garde| [6, 7]|    5.0|This wa

# Part 1: RDDs

# Part 2: 

In [4]:
# Casefolding
df = df.withColumn("reviewText", lower(col("reviewText")))

In [10]:
# Сonverting category to numeric
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

# Creating the pipeline:

# 1. tokenize
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern=r'\s+|\t+|\d+|[(){}.!?,;:+=-_"\`~#@&*%€$§\\/]+', gaps=True)

# 3. stopwords removal
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered_words")


# 4. TF-IDF calculation with HashingTF
hashingTF = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="rawFeatures", numFeatures=20)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# 5. Chi-square
selector = ChiSqSelector(numTopFeatures=2000, featuresCol=idf.getOutputCol(),
                         outputCol="selectedFeatures", labelCol=indexer.getOutputCol())



# Part 3:

In [15]:
# 6. Normalizing
normalizer = Normalizer(inputCol=selector.getOutputCol(), outputCol="normFeatures")

# 7. Creating SVM classificator
svm = LinearSVC(featuresCol=normalizer.getOutputCol(), labelCol="categoryIndex")

# 8. Using One-vs-Rest for multiclass classification
ovr = OneVsRest(classifier=svm)


pipeline = Pipeline(stages=[indexer, tokenizer, stopwords_remover, hashingTF, idf, selector, normalizer, svm, ovr])

# Splitting the data
train_df, test_df, validation_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)


# Show the count of each set
print("Training set count:", train_df.count())
print("Test set count:", test_df.count())
print("Validation set count:", validation_df.count())

Training set count: 55401
Test set count: 11621
Validation set count: 11807


In [ ]:
# Creating a grid of parameters
paramGrid = (ParamGridBuilder()
             .addGrid(selector.numTopFeatures, [2000, 500])  # number of features selected
             .addGrid(svm.regParam, [0.01, 0.1, 1.0])  # Regularization parameter
             .addGrid(svm.maxIter, [10, 50])  # Maximum number of iterations
             .addGrid(svm.standardization, [True, False]) # Standardization
             .build())


# Creating a Cross Validator for Grid search
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol=indexer.getOutputCol(), metricName="f1"),
                          numFolds=3)

# Model training
model = crossval.fit(train_df)

# Predictions on test data
predictions = model.transform(test_df)

# Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)

print(f"F1 Score: {f1_score}")

# Output of the result
# terms = result.select("selectedFeatures").collect()
# with open("output_ds.txt", "w") as f:
#     for term in terms:
#         f.write(f"{term}\n")

# Stopping Spark
spark.stop()